# 数据挖掘-预测贷款用户是否逾期

## Task1：数据分析

#### 数据说明：
数据集是金融数据（非原始数据，已经处理过了），预测贷款用户是否会逾期

表格中 "status" 是结果标签：0表示未逾期，1表示逾期

### 1 数据类型的分析 

#### ● 导入宏包

In [31]:
import numpy as np
import pandas as pd
import seaborn as sns  #Seaborn是基于matplotlib的图形可视化python包
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

#作图默认无法显示中文（变成小方格）
import matplotlib as mpl
mpl.rcParams['font.sans-serif']=[u'SimHei']    #设置中文黑体
mpl.rcParams['axes.unicode_minus']=False       #更改字体导致显示不出正负号，设置这个显示正常

#### ● 导入数据

In [32]:
data = pd.read_csv('F:/project/data.csv',engine = 'python')
data.head()

,Unnamed: 0,custid,trade_no,bank_card_no,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
0,5,2791858,20180507115231274000000023057383,卡号1,0.01,0.99,0,0.90,0.55,0.313,...,2900.0,1688.0,1200.0,75.0,1.0,2.0,1200.0,1200.0,12.0,18.0
1,10,534047,20180507121002192000000023073000,卡号1,0.02,0.94,2000,1.28,1.00,0.458,...,3500.0,1758.0,15100.0,80.0,5.0,6.0,22800.0,9360.0,4.0,2.0
2,12,2849787,20180507125159718000000023114911,卡号1,0.04,0.96,0,1.00,1.00,0.114,...,1600.0,1250.0,4200.0,87.0,1.0,1.0,4200.0,4200.0,2.0,6.0
3,13,1809708,20180507121358683000000388283484,卡号1,0.00,0.96,2000,0.13,0.57,0.777,...,3200.0,1541.0,16300.0,80.0,5.0,5.0,30000.0,12180.0,2.0,4.0
4,14,2499829,20180507115448545000000388205844,卡号1,0.01,0.99,0,0.46,1.00,0.175,...,2300.0,1630.0,8300.0,79.0,2.0,2.0,8400.0,8250.0,22.0,120.0


#### ● 数据类型

In [33]:
data.info()   #依次为变量名，数据个数，是否有缺失，数据类型

print()
print('数据条数：',data.shape[0])  #输出数组的行和列数 .shape[0] 只输出行数  .shape[1]  只输出列数

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4754 entries, 0 to 4753
Data columns (total 90 columns):
Unnamed: 0                                    4754 non-null int64
custid                                        4754 non-null int64
trade_no                                      4754 non-null object
bank_card_no                                  4754 non-null object
low_volume_percent                            4752 non-null float64
middle_volume_percent                         4752 non-null float64
take_amount_in_later_12_month_highest         4754 non-null int64
trans_amount_increase_rate_lately             4751 non-null float64
trans_activity_month                          4752 non-null float64
trans_activity_day                            4752 non-null float64
transd_mcc                                    4752 non-null float64
trans_days_interval_filter                    4746 non-null float64
trans_days_interval                           4752 non-null float64
regional_mobility

#### ● 数据基本统计量描述

In [34]:
data.describe()

,Unnamed: 0,custid,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
count,4754.000000,4.754000e+03,4752.000000,4752.000000,4754.000000,4751.000000,4752.000000,4752.000000,4752.000000,4746.000000,...,4457.000000,4457.000000,4457.000000,4457.000000,4457.000000,4457.000000,4457.000000,4457.000000,4450.000000,4457.000000
mean,6008.414178,1.690993e+06,0.021806,0.901294,1940.197728,14.160674,0.804411,0.365425,17.502946,29.029920,...,3390.038142,1820.357864,9187.009199,76.042630,4.732331,5.227507,16153.690823,8007.696881,24.112809,55.181512
std,3452.071428,1.034235e+06,0.041527,0.144856,3923.971494,694.180473,0.196920,0.170196,4.475616,22.722432,...,1474.206546,583.418291,7371.257043,14.536819,2.974596,3.409292,14301.037628,5679.418585,37.725724,53.486408
min,5.000000,1.140000e+02,0.000000,0.000000,0.000000,0.000000,0.120000,0.033000,2.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.000000,-2.000000
25%,3106.000000,7.593358e+05,0.010000,0.880000,0.000000,0.615000,0.670000,0.233000,15.000000,16.000000,...,2300.000000,1535.000000,4800.000000,77.000000,2.000000,3.000000,7800.000000,4737.000000,5.000000,10.000000
50%,6006.500000,1.634942e+06,0.010000,0.960000,500.000000,0.970000,0.860000,0.350000,17.000000,23.000000,...,3100.000000,1810.000000,7700.000000,79.000000,4.000000,5.000000,13800.000000,7050.000000,14.000000,36.000000
75%,8999.000000,2.597905e+06,0.020000,0.990000,2000.000000,1.600000,1.000000,0.480000,20.000000,32.000000,...,4300.000000,2100.000000,11700.000000,80.000000,7.000000,7.000000,20400.000000,10000.000000,24.000000,91.000000
max,11992.000000,4.004694e+06,1.000000,1.000000,68000.000000,47596.740000,1.000000,0.941000,42.000000,285.000000,...,10000.000000,6900.000000,87100.000000,87.000000,18.000000,20.000000,266400.000000,82800.000000,360.000000,323.000000


#### ● 结果标签分布情况

In [35]:
data['status'].value_counts()

0    3561
1    1193
Name: status, dtype: int64

### 2 无关特征删除 

### 3 数据类型转换

#### ● object 类型特征
先使用将类别属性转换成一个标量值的方式，哑编码（独热one-hot编码方式）的方式暂时不会

In [36]:
data['reg_preference_for_trad'].unique()   #查看字段里的分类内容

array(['一线城市', '三线城市', '境外', '二线城市', '其他城市', nan], dtype=object)

In [37]:
dic = {'一线城市': 1,
       '二线城市': 2,
       '三线城市': 3,
       '其他城市': 4,
       '境外': 0}
data.reg_preference_for_trad = data.reg_preference_for_trad.map(dic)
#data['reg_preference_for_trad1'] = data['reg_preference_for_trad'].map(dic)   #不明白这样写为啥不对，都是NAN
data.reg_preference_for_trad

0       1.0
1       1.0
2       1.0
3       3.0
4       1.0
5       3.0
6       1.0
7       1.0
8       3.0
9       1.0
10      1.0
11      1.0
12      3.0
13      1.0
14      1.0
15      1.0
16      0.0
17      1.0
18      0.0
19      1.0
20      1.0
21      1.0
22      0.0
23      3.0
24      1.0
25      0.0
26      1.0
27      3.0
28      1.0
29      1.0
       ... 
4724    1.0
4725    1.0
4726    0.0
4727    1.0
4728    1.0
4729    1.0
4730    3.0
4731    3.0
4732    1.0
4733    2.0
4734    0.0
4735    1.0
4736    1.0
4737    1.0
4738    1.0
4739    3.0
4740    1.0
4741    1.0
4742    1.0
4743    3.0
4744    1.0
4745    3.0
4746    1.0
4747    0.0
4748    1.0
4749    1.0
4750    1.0
4751    1.0
4752    1.0
4753    1.0
Name: reg_preference_for_trad, Length: 4754, dtype: float64

#### ● 时间格式